In [1]:
USE AdventureWorks2014;
GO

Commands completed successfully.

Total execution time: 00:00:00.051

In [2]:
-- 7-1. Computing an Average

SELECT ProductID,
AVG(Rating) AS AvgRating
FROM Production.ProductReview
GROUP BY ProductID;

(3 rows affected)

Total execution time: 00:00:00.165

ProductID,AvgRating
709,5
798,5
937,3


In [3]:
-- 7-2. Counting the Rows in a Group

SELECT TOP (5)
    Shelf,
    COUNT(ProductID) AS ProductCount,
    COUNT_BIG(ProductID) AS ProductCountBig
FROM Production.ProductInventory
GROUP BY Shelf
ORDER BY Shelf;

(5 rows affected)

Total execution time: 00:00:00.053

Shelf,ProductCount,ProductCountBig
A,81,81
B,36,36
C,55,55
D,50,50
E,85,85


In [4]:
-- 7-3. Summing the Values in a Group

SELECT TOP (5)
    AccountNumber,
    SUM(TotalDue) AS TotalDueByAccountNumber
FROM Sales.SalesOrderHeader
GROUP BY AccountNumber
ORDER BY AccountNumber;

(5 rows affected)

Total execution time: 00:00:00.109

AccountNumber,TotalDueByAccountNumber
10-4020-000001,95924.0197
10-4020-000002,28309.9672
10-4020-000003,407563.0075
10-4020-000004,660645.9404
10-4020-000005,97031.2173


In [5]:
-- 7-4. Finding the High and Low Values in a Group

SELECT MIN(Rating) MinRating,
    MAX(Rating) MaxRating
FROM Production.ProductReview;

(1 row affected)

Total execution time: 00:00:00.028

MinRating,MaxRating
2,5


In [6]:
-- 7-5. Detecting Changes in a Table

SELECT StudentId,
    CHECKSUM_AGG(Grade) AS GradeChecksumAgg
FROM (VALUES (1, 100),
            (1, 100),           
            (1, 100),
            (1, 99),
            (1, 99),
            (1, 98),
            (1, 98),
            (1, 95),
            (1, 95),
            (1, 95)
    ) dt (StudentId, Grade)
GROUP BY StudentID;
SELECT StudentId,
    CHECKSUM_AGG(Grade) AS GradeChecksumAgg
FROM (VALUES (1, 100),
            (1, 100),
            (1, 100),
            (1, 99),
            (1, 99),
            (1, 98),
            (1, 98),
            (1, 95),
            (1, 95),
            (1, 90)
    ) dt (StudentId, Grade)
GROUP BY StudentID;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.070

StudentId,GradeChecksumAgg
1,59


StudentId,GradeChecksumAgg
1,62


In [7]:
-- 7-6. Finding the Statistical Variance in the Values of a Column

SELECT VAR(TaxAmt) AS Variance_Sample,
    VARP(TaxAmt) AS Variance_EntirePopulation
FROM Sales.SalesOrderHeader;

(1 row affected)

Total execution time: 00:00:00.086

Variance_Sample,Variance_EntirePopulation
1177342.572774014,1177305.1552442897


In [8]:
-- 7-7. Finding the Standard Deviation in the Values of a Column

SELECT STDEV(UnitPrice) AS StandDevUnitPrice,
    STDEVP(UnitPrice) AS StandDevPopUnitPrice
FROM Sales.SalesOrderDetail;

(1 row affected)

Total execution time: 00:00:04.811

StandDevUnitPrice,StandDevPopUnitPrice
751.8850807729535,751.8819819218847


In [9]:
-- 7-8. Calculating Totals Based Upon the Prior Row

SELECT AccountId,
    TranDate,
    TranAmt,
    -- running total of all transactions
    RunTotalAmt = SUM(TranAmt) OVER (PARTITION BY AccountId ORDER BY TranDate)
FROM #Transactions AS t
ORDER BY AccountI

: Msg 208, Level 16, State 0, Line 3
Invalid object name '#Transactions'.

Total execution time: 00:00:00.031

In [10]:
-- 7-9. Calculating Totals Based Upon a Subset of Rows

SELECT AccountId,
       TranDate,
       TranAmt,
       -- average of the current and previous 2 transactions
       SlideAvg = AVG(TranAmt)
                  OVER (PARTITION BY AccountId
                        ORDER BY TranDate
                            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW),
       -- total # of the current and previous 2 transactions
       SlideQty = COUNT(*)
                  OVER (PARTITION BY AccountId
                        ORDER BY TranDate
                            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW),
       -- smallest of the current and previous 2 transactions
       SlideMin = MIN(TranAmt)
                  OVER (PARTITION BY AccountId
                        ORDER BY TranDate
                            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW),
       -- largest of the current and previous 2 transactions
       SlideMax = MAX(TranAmt)
                  OVER (PARTITION BY AccountId
                        ORDER BY TranDate
                            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW),
       -- total of the current and previous 2 transactions
       SlideTotal = SUM(TranAmt)
                    OVER (PARTITION BY AccountId
                        ORDER BY TranDate
                            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
FROM #Transactions AS t
ORDER BY AccountId,
      TranDate;

: Msg 208, Level 16, State 0, Line 3
Invalid object name '#Transactions'.

Total execution time: 00:00:00.005

In [11]:
-- 7-10. Using a Logical Window

DECLARE @Test TABLE
    (
        RowID INT IDENTITY,
        FName VARCHAR(20),
        Salary SMALLINT
    );
INSERT INTO @Test (FName, Salary)
VALUES ('George', 800),
    ('Sam', 950),
    ('Diane', 1100),
    ('Nicholas', 1250),
    ('Samuel', 1250),
    ('Patricia', 1300),
    ('Brian', 1500),
    ('Thomas', 1600),
    ('Fran', 2450),
    ('Debbie', 2850),
    ('Mark', 2975),
    ('James', 3000),
    ('Cynthia', 3000),
    ('Christopher', 5000);
SELECT RowID,
    FName,
    Salary,
    SumByRows = SUM(Salary)
        OVER (ORDER BY Salary
            ROWS UNBOUNDED PRECEDING),
    SumByRange = SUM(Salary)
        OVER (ORDER BY Salary
            RANGE UNBOUNDED PRECEDING)
FROM @Test
ORDER BY RowID;

(14 rows affected)

(14 rows affected)

Total execution time: 00:00:00.101

RowID,FName,Salary,SumByRows,SumByRange
1,George,800,800,800
2,Sam,950,1750,1750
3,Diane,1100,2850,2850
4,Nicholas,1250,4100,5350
5,Samuel,1250,5350,5350
6,Patricia,1300,6650,6650
7,Brian,1500,8150,8150
8,Thomas,1600,9750,9750
9,Fran,2450,12200,12200
10,Debbie,2850,15050,15050


In [12]:
-- 7-11. Generating an Incrementing Row Number

SELECT TOP 10
    AccountNumber,
    OrderDate,
    TotalDue,
    ROW_NUMBER() OVER (PARTITION BY AccountNumber ORDER BY OrderDate) AS RN
FROM Sales.SalesOrderHeader
ORDER BY AccountNumber;

(10 rows affected)

Total execution time: 00:00:00.219

AccountNumber,OrderDate,TotalDue,RN
10-4020-000001,2011-07-01 00:00:00.000,12381.0798,1
10-4020-000001,2011-10-01 00:00:00.000,22152.2446,2
10-4020-000001,2012-01-01 00:00:00.000,31972.1684,3
10-4020-000001,2012-03-30 00:00:00.000,29418.5269,4
10-4020-000002,2012-06-30 00:00:00.000,8727.1055,1
10-4020-000002,2012-09-30 00:00:00.000,4682.6908,2
10-4020-000002,2012-12-31 00:00:00.000,1485.9180,3
10-4020-000002,2013-03-30 00:00:00.000,1668.3751,4
10-4020-000002,2013-06-30 00:00:00.000,3478.1096,5
10-4020-000002,2013-09-30 00:00:00.000,3941.9843,6


In [13]:
-- 7-12. Returning Rows by Rank

SELECT BusinessEntityID,
    QuotaDate,
    SalesQuota,
    RANK() OVER (ORDER BY SalesQuota DESC) AS RANK
FROM Sales.SalesPersonQuotaHistory
WHERE SalesQuota BETWEEN 266000.00 AND 319000.00;

(10 rows affected)

Total execution time: 00:00:01.503

BusinessEntityID,QuotaDate,SalesQuota,RANK
280,2013-05-30 00:00:00.000,319000.0000,1
284,2013-02-28 00:00:00.000,304000.0000,2
280,2012-02-29 00:00:00.000,301000.0000,3
282,2012-11-30 00:00:00.000,288000.0000,4
283,2013-02-28 00:00:00.000,284000.0000,5
284,2012-11-30 00:00:00.000,281000.0000,6
278,2013-11-30 00:00:00.000,280000.0000,7
283,2011-12-01 00:00:00.000,280000.0000,7
283,2012-02-29 00:00:00.000,267000.0000,9
278,2011-12-01 00:00:00.000,266000.0000,10


In [14]:
-- 7-13. Returning Rows by Rank Without Gaps

SELECT BusinessEntityID,
    QuotaDate,
    SalesQuota,
    DENSE_RANK() OVER (ORDER BY SalesQuota DESC) AS DENSERANK
FROM Sales.SalesPersonQuotaHistory
WHERE SalesQuota BETWEEN 266000.00 AND 319000.00;

(10 rows affected)

Total execution time: 00:00:00.039

BusinessEntityID,QuotaDate,SalesQuota,DENSERANK
280,2013-05-30 00:00:00.000,319000.0000,1
284,2013-02-28 00:00:00.000,304000.0000,2
280,2012-02-29 00:00:00.000,301000.0000,3
282,2012-11-30 00:00:00.000,288000.0000,4
283,2013-02-28 00:00:00.000,284000.0000,5
284,2012-11-30 00:00:00.000,281000.0000,6
278,2013-11-30 00:00:00.000,280000.0000,7
283,2011-12-01 00:00:00.000,280000.0000,7
283,2012-02-29 00:00:00.000,267000.0000,8
278,2011-12-01 00:00:00.000,266000.0000,9


In [15]:
-- 7-14. Sorting Rows into Buckets

SELECT BusinessEntityID,
    QuotaDate,
    SalesQuota,
    NTILE(4) OVER (ORDER BY SalesQuota DESC) AS [NTILE]
FROM Sales.SalesPersonQuotaHistory
WHERE SalesQuota BETWEEN 266000.00 AND 319000.00;

(10 rows affected)

Total execution time: 00:00:00.028

BusinessEntityID,QuotaDate,SalesQuota,NTILE
280,2013-05-30 00:00:00.000,319000.0000,1
284,2013-02-28 00:00:00.000,304000.0000,1
280,2012-02-29 00:00:00.000,301000.0000,1
282,2012-11-30 00:00:00.000,288000.0000,2
283,2013-02-28 00:00:00.000,284000.0000,2
284,2012-11-30 00:00:00.000,281000.0000,2
278,2013-11-30 00:00:00.000,280000.0000,3
283,2011-12-01 00:00:00.000,280000.0000,3
283,2012-02-29 00:00:00.000,267000.0000,4
278,2011-12-01 00:00:00.000,266000.0000,4


In [16]:
-- 7-15. Grouping Logically Consecutive Rows Together

DECLARE @RFID_Location TABLE (
    TagId INTEGER,
    Location VARCHAR(25),
    SensorDate DATETIME);
INSERT INTO @RFID_Location
    (TagId, Location, SensorDate)
VALUES (1, 'Room1', '2012-01-10T08:00:01'),
    (1, 'Room1', '2012-01-10T08:18:32'),
    (1, 'Room2', '2012-01-10T08:25:42'),
    (1, 'Room3', '2012-01-10T09:52:48'),
    (1, 'Room2', '2012-01-10T10:05:22'),
    (1, 'Room3', '2012-01-10T11:22:15'),
    (1, 'Room4', '2012-01-10T14:18:58'),
    (2, 'Room1', '2012-01-10T08:32:18'),
    (2, 'Room1', '2012-01-10T08:51:53'),
    (2, 'Room2', '2012-01-10T09:22:09'),
    (2, 'Room1', '2012-01-10T09:42:17'),
    (2, 'Room1', '2012-01-10T09:59:16'),
    (2, 'Room2', '2012-01-10T10:35:18'),
    (2, 'Room3', '2012-01-10T11:18:42'),
    (2, 'Room4', '2012-01-10T15:22:18');

(15 rows affected)

Total execution time: 00:00:00.038

In [17]:
-- 7-16. Accessing Values from Other Rows

WITH cte AS
(
SELECT DATEPART(QUARTER, OrderDate) AS Qtr,
    DATEPART(YEAR, OrderDate) AS Yr,
    TotalDue
FROM Sales.SalesOrderHeader
), cteAgg AS
(
SELECT Yr,
    Qtr,
    SUM(TotalDue) AS TotalDue
FROM cte
GROUP BY Yr, Qtr
)
SELECT Yr,
    Qtr,
    TotalDue,
    TotalDue - LAG(TotalDue, 1, NULL)
        OVER (ORDER BY Yr, Qtr) AS DeltaPriorQtr,
    TotalDue - LAG(TotalDue, 4, NULL)
        OVER (ORDER BY Yr, Qtr) AS DeltaPriorYrQtr
FROM cteAgg
ORDER BY Yr, Qtr;

(13 rows affected)

Total execution time: 00:00:00.116

Yr,Qtr,TotalDue,DeltaPriorQtr,DeltaPriorYrQtr
2011,2,1074117.4188,NULL,NULL
2011,3,5647550.6633,4573433.2445,NULL
2011,4,7434031.4429,1786480.7796,NULL
2012,1,9443736.8161,2009705.3732,NULL
2012,2,9935495.1729,491758.3568,8861377.7541
2012,3,10164406.8281,228911.6552,4516856.1648
2012,4,8132061.4949,-2032345.3332,698030.0520
2013,1,8771886.3577,639824.8628,-671850.4584
2013,2,12225061.3830,3453175.0253,2289566.2101
2013,3,14339319.1851,2114257.8021,4174912.3570


In [18]:
-- 7-17. Accessing the First or Last Value from a Partition

SELECT CustomerID,
    CUME_DIST()
    OVER (PARTITION BY CustomerID
        ORDER BY TotalDue) AS CumeDistOrderTotalDue,
    PERCENT_RANK()
    OVER (PARTITION BY CustomerID
        ORDER BY TotalDue) AS PercentRankOrderTotalDue
FROM Sales.SalesOrderHeader
ORDER BY CustomerID;

(31465 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:01.402

CustomerID,CumeDistOrderTotalDue,PercentRankOrderTotalDue
11000,0.3333333333333333,0
11000,0.6666666666666666,0.5
11000,1,1
11001,0.3333333333333333,0
11001,0.6666666666666666,0.5
11001,1,1
11002,0.3333333333333333,0
11002,0.6666666666666666,0.5
11002,1,1
11003,0.3333333333333333,0


In [19]:
-- 7-18. Calculating the Relative Position or Rank of a Value in a Set of Values

SELECT CustomerID,
    CUME_DIST()
    OVER (PARTITION BY CustomerID
        ORDER BY TotalDue) AS CumeDistOrderTotalDue,
    PERCENT_RANK()
    OVER (PARTITION BY CustomerID
        ORDER BY TotalDue) AS PercentRankOrderTotalDue
FROM Sales.SalesOrderHeader
ORDER BY CustomerID;

(31465 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:01.258

CustomerID,CumeDistOrderTotalDue,PercentRankOrderTotalDue
11000,0.3333333333333333,0
11000,0.6666666666666666,0.5
11000,1,1
11001,0.3333333333333333,0
11001,0.6666666666666666,0.5
11001,1,1
11002,0.3333333333333333,0
11002,0.6666666666666666,0.5
11002,1,1
11003,0.3333333333333333,0


In [20]:
--7-19. Calculating Continuous or Discrete Percentiles

DECLARE @Employees TABLE
    (
    EmplId INT PRIMARY KEY CLUSTERED,
    DeptId INT,
    Salary NUMERIC(8, 2)
    );
INSERT INTO @Employees
VALUES (1, 1, 10000),
    (2, 1, 11000),
    (3, 1, 12000),
    (4, 2, 25000),
    (5, 2, 35000),
    (6, 2, 75000),
    (7, 2, 100000);
SELECT EmplId,
    DeptId,
    Salary,
    PERCENTILE_CONT(0.5)
        WITHIN GROUP (ORDER BY Salary ASC)
        OVER (PARTITION BY DeptId) AS MedianCont,
    PERCENTILE_DISC(0.5)
        WITHIN GROUP (ORDER BY Salary ASC)
        OVER (PARTITION BY DeptId) AS MedianDisc,
    PERCENTILE_CONT(0.75)
        WITHIN GROUP (ORDER BY Salary ASC)
        OVER (PARTITION BY DeptId) AS Percent75Cont,
    PERCENTILE_DISC(0.75)
        WITHIN GROUP (ORDER BY Salary ASC)
        OVER (PARTITION BY DeptId) AS Percent75Disc,
    CUME_DIST()
        OVER (PARTITION BY DeptId
            ORDER BY Salary) AS CumeDist
FROM @Employees
ORDER BY DeptId, EmplId;

(7 rows affected)

(7 rows affected)

Total execution time: 00:00:00.090

EmplId,DeptId,Salary,MedianCont,MedianDisc,Percent75Cont,Percent75Disc,CumeDist
1,1,10000.00,11000,11000.00,11500,12000.00,0.3333333333333333
2,1,11000.00,11000,11000.00,11500,12000.00,0.6666666666666666
3,1,12000.00,11000,11000.00,11500,12000.00,1
4,2,25000.00,55000,35000.00,81250,75000.00,0.25
5,2,35000.00,55000,35000.00,81250,75000.00,0.5
6,2,75000.00,55000,35000.00,81250,75000.00,0.75
7,2,100000.00,55000,35000.00,81250,75000.00,1


In [21]:
-- 7-20. Assigning Sequences in a Specified Order

IF EXISTS (SELECT 1
        FROM sys.sequences AS seq
            JOIN sys.schemas AS sch
                ON seq.schema_id = sch.schema_id
        WHERE sch.name = 'dbo'
        AND seq.name = 'CH7Sequence')
    DROP SEQUENCE dbo.CH7Sequence;
CREATE SEQUENCE dbo.CH7Sequence AS INTEGER START WITH 1;
DECLARE @ClassRank TABLE
    (
    StudentID TINYINT,
    Grade TINYINT,
    SeqNbr INTEGER
    );
INSERT INTO @ClassRank (StudentId, Grade, SeqNbr)
SELECT StudentId,
    Grade,
    NEXT VALUE FOR dbo.CH7Sequence OVER (ORDER BY Grade ASC)
FROM (VALUES (1, 100),
    (2, 95),
    (3, 85),
    (4, 100),
    (5, 99),
    (6, 98),
    (7, 95),
    (8, 90),
    (9, 89),
    (10, 89),
    (11, 85),
    (12, 82)) dt(StudentId, Grade);
SELECT StudentId, Grade, SeqNbr
FROM @ClassRank;

(12 rows affected)

(12 rows affected)

Total execution time: 00:00:00.091

StudentId,Grade,SeqNbr
12,82,1
3,85,2
11,85,3
10,89,4
9,89,5
8,90,6
7,95,7
2,95,8
6,98,9
5,99,10
